In [1]:
import sys
import os
print(sys.executable)
print(os.environ['PATH'])

import numpy as np
import pandas as pd
import requests
import bs4
import fastnumbers
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import json
#from tidylib import tidy_document # for tidying incorrect html

sns.set_style('whitegrid')
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

C:\Users\jogle\Anaconda3\python.exe
C:\Users\jogle\Anaconda3;C:\Users\jogle\Anaconda3\Library\mingw-w64\bin;C:\Users\jogle\Anaconda3\Library\usr\bin;C:\Users\jogle\Anaconda3\Library\bin;C:\Users\jogle\Anaconda3\Scripts;C:\Program Files (x86)\Common Files\Oracle\Java\javapath;C:\Program Files (x86)\Intel\Intel(R) Management Engine Components\iCLS\;C:\Program Files\Intel\Intel(R) Management Engine Components\iCLS\;C:\Windows\system32;C:\Windows;C:\Windows\System32\Wbem;C:\Windows\System32\WindowsPowerShell\v1.0\;C:\Program Files (x86)\Intel\Intel(R) Management Engine Components\DAL;C:\Program Files\Intel\Intel(R) Management Engine Components\DAL;C:\Program Files (x86)\Intel\Intel(R) Management Engine Components\IPT;C:\Program Files\Intel\Intel(R) Management Engine Components\IPT;C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common;C:\Program Files\Intel\WiFi\bin\;C:\Program Files\Common Files\Intel\WirelessCommon\;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\Syste

In [2]:
############ To get Ticker List ################

data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
table = data[0]
sliced_table = table[1:]
header = table.iloc[0]
corrected_table = sliced_table.rename(columns=header)
company_List = ['3M Company','Adobe Systems Inc','Allstate Corp','Amazon.com Inc.','Apple Inc.','American Express Co','JPMorgan Chase & Co.','Visa Inc.','Xilinx','Facebook, Inc.']
corrected_table = corrected_table[corrected_table.Security.isin(company_List)]
#print(len(company_List))
#corrected_table
tickers = corrected_table['Symbol'].tolist()
print (tickers)
s,s2 = 'https://www.worldtradingdata.com/api/v1/stock?symbol=','https://www.worldtradingdata.com/api/v1/stock?symbol='
for i in range(0,len(tickers)-7):
    s = s + tickers[i] + ','
s = s + tickers[len(tickers)-6] + '&api_token=p4nU2OENDDJfkKP6TePnfKAgI4W5sz2GQmsVBehWiAjwLN6srpgbqp8lfiaV'
s
for i in range(len(tickers)-5,len(tickers)-1):
    s2 = s2 + tickers[i] + ','
s2 = s2 + tickers[-1] + '&api_token=p4nU2OENDDJfkKP6TePnfKAgI4W5sz2GQmsVBehWiAjwLN6srpgbqp8lfiaV'
s2

['MMM', 'ADBE', 'ALL', 'AMZN', 'AXP', 'AAPL', 'FB', 'JPM', 'V', 'XLNX']


'https://www.worldtradingdata.com/api/v1/stock?symbol=MMM,ADBE,ALL,AXP&api_token=p4nU2OENDDJfkKP6TePnfKAgI4W5sz2GQmsVBehWiAjwLN6srpgbqp8lfiaV'

'https://www.worldtradingdata.com/api/v1/stock?symbol=AAPL,FB,JPM,V,XLNX&api_token=p4nU2OENDDJfkKP6TePnfKAgI4W5sz2GQmsVBehWiAjwLN6srpgbqp8lfiaV'

In [3]:
################# Call get method to take response from URL ###################

#response = requests.get("https://www.worldtradingdata.com/api/v1/stock?symbol=AAPL,MSFT,HSBA.L&api_token=demo", timeout=240)
response = requests.get(s, timeout=240)
response2 = requests.get(s2, timeout=240)
#response.status_code
response.content
response2.content

b'{"symbols_requested":4,"symbols_returned":4,"data":[{"symbol":"ADBE","name":"Adobe Inc.","currency":"USD","price":"275.45","price_open":"279.22","day_high":"280.17","day_low":"275.25","52_week_high":"280.17","52_week_low":"204.95","day_change":"-3.04","change_pct":"-1.09","close_yesterday":"278.49","market_cap":"134406111232","volume":"1632885","volume_avg":"1929900","shares":"487951008","stock_exchange_long":"NASDAQ Stock Exchange","stock_exchange_short":"NASDAQ","timezone":"EDT","timezone_name":"America/New_York","gmt_offset":"-14400","last_trade_time":"2019-04-24 16:00:00"},{"symbol":"ALL","name":"The Allstate Corporation","currency":"USD","price":"96.41","price_open":"97.08","day_high":"97.44","day_low":"96.38","52_week_high":"102.73","52_week_low":"77.00","day_change":"-0.88","change_pct":"-0.90","close_yesterday":"97.29","market_cap":"32102408192","volume":"1410055","volume_avg":"1561316","shares":"332977984","stock_exchange_long":"New York Stock Exchange","stock_exchange_short

b'{"symbols_requested":5,"symbols_returned":5,"data":[{"symbol":"AAPL","name":"Apple Inc.","currency":"USD","price":"207.16","price_open":"207.36","day_high":"208.48","day_low":"207.05","52_week_high":"233.47","52_week_low":"142.00","day_change":"-0.32","change_pct":"-0.15","close_yesterday":"207.48","market_cap":"976817422336","volume":"15166983","volume_avg":"23180683","shares":"4715279872","stock_exchange_long":"NASDAQ Stock Exchange","stock_exchange_short":"NASDAQ","timezone":"EDT","timezone_name":"America/New_York","gmt_offset":"-14400","last_trade_time":"2019-04-24 16:00:01"},{"symbol":"FB","name":"Facebook, Inc.","currency":"USD","price":"182.58","price_open":"184.49","day_high":"185.14","day_low":"181.65","52_week_high":"218.62","52_week_low":"123.02","day_change":"-1.20","change_pct":"-0.65","close_yesterday":"183.78","market_cap":"521377284096","volume":"23758514","volume_avg":"12786333","shares":"2403460096","stock_exchange_long":"NASDAQ Stock Exchange","stock_exchange_short

In [4]:
############### Convert the content to json ################
content = response.json()
content2 = response2.json()
#content.keys()
#content = content['data']
#type(content)
#content = json.dumps(content)
keys = content['data'][0].keys()
keys2 = content2['data'][0].keys()
#keys
content['data']
content2['data']

[{'symbol': 'ADBE',
  'name': 'Adobe Inc.',
  'currency': 'USD',
  'price': '275.45',
  'price_open': '279.22',
  'day_high': '280.17',
  'day_low': '275.25',
  '52_week_high': '280.17',
  '52_week_low': '204.95',
  'day_change': '-3.04',
  'change_pct': '-1.09',
  'close_yesterday': '278.49',
  'market_cap': '134406111232',
  'volume': '1632885',
  'volume_avg': '1929900',
  'shares': '487951008',
  'stock_exchange_long': 'NASDAQ Stock Exchange',
  'stock_exchange_short': 'NASDAQ',
  'timezone': 'EDT',
  'timezone_name': 'America/New_York',
  'gmt_offset': '-14400',
  'last_trade_time': '2019-04-24 16:00:00'},
 {'symbol': 'ALL',
  'name': 'The Allstate Corporation',
  'currency': 'USD',
  'price': '96.41',
  'price_open': '97.08',
  'day_high': '97.44',
  'day_low': '96.38',
  '52_week_high': '102.73',
  '52_week_low': '77.00',
  'day_change': '-0.88',
  'change_pct': '-0.90',
  'close_yesterday': '97.29',
  'market_cap': '32102408192',
  'volume': '1410055',
  'volume_avg': '1561316'

[{'symbol': 'AAPL',
  'name': 'Apple Inc.',
  'currency': 'USD',
  'price': '207.16',
  'price_open': '207.36',
  'day_high': '208.48',
  'day_low': '207.05',
  '52_week_high': '233.47',
  '52_week_low': '142.00',
  'day_change': '-0.32',
  'change_pct': '-0.15',
  'close_yesterday': '207.48',
  'market_cap': '976817422336',
  'volume': '15166983',
  'volume_avg': '23180683',
  'shares': '4715279872',
  'stock_exchange_long': 'NASDAQ Stock Exchange',
  'stock_exchange_short': 'NASDAQ',
  'timezone': 'EDT',
  'timezone_name': 'America/New_York',
  'gmt_offset': '-14400',
  'last_trade_time': '2019-04-24 16:00:01'},
 {'symbol': 'FB',
  'name': 'Facebook, Inc.',
  'currency': 'USD',
  'price': '182.58',
  'price_open': '184.49',
  'day_high': '185.14',
  'day_low': '181.65',
  '52_week_high': '218.62',
  '52_week_low': '123.02',
  'day_change': '-1.20',
  'change_pct': '-0.65',
  'close_yesterday': '183.78',
  'market_cap': '521377284096',
  'volume': '23758514',
  'volume_avg': '12786333

In [6]:
### Witing JSON data into CSV ##################
#csvwriter = csv.writer(first_csv)
with open('D:\sum\Its Study Time\Academic\M.S\Spring_2019\Machine Learning\Project\Sumedh_Work\\first_csv.csv', 'a', newline='') as first_csv:
    csvwriter = csv.DictWriter(first_csv, keys,extrasaction='ignore')
    #csvwriter.writeheader()
    csvwriter.writerows(content['data'])
    csvwriter.writerows(content2['data'])

In [7]:
############# Play using pandas ################
data = pd.read_csv('D:\sum\Its Study Time\Academic\M.S\Spring_2019\Machine Learning\Project\Sumedh_Work\\first_csv.csv')
#data.head
#data.drop()
data.shape

(12959, 22)